In [ ]:
%reload_ext tgpy
import sys
# Hay que ejecutar esta línea antes de importar el módulo.
sys.path.append("C:/Users/poker/PycharmProjects/tgpy")
# Ahora se puede importar el módulo.
import tgpy as tg

In [ ]:
npriors = 3 # 1, 2, 3, 4
ngroups = 4 # any positive int
nparams = 500 # number of chains
nitersSGD = 300

### Create Priors

In [ ]:
prior0 = tg.TgPrior('prior0', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                   low=-4, high=4, alpha=5, beta=5) # simetrico
prior1 = tg.TgPrior('prior1', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                   low=-1, high=10, alpha=2, beta=3) # masa a la izquierda
prior2 = tg.TgPrior('prior2', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                   low=-10, high=1, alpha=2, beta=1) # masa a la derecha
prior3 = tg.TgPrior('prior3', ['g{}'.format(i) for i in range(ngroups)], dim=nparams, 
                   low=-4, high=4, alpha=0.5, beta=0.5) # doble masa

                    
if npriors == 1: 
    k = tg.DummyKernel(prior0)
elif npriors == 2:
    k = tg.DummyKernel(prior0, prior1)
elif npriors == 3:
    k = tg.DummyKernel(prior0, prior1, prior2) 
else:
    k = tg.DummyKernel(prior0, prior1, prior2, prior3) 


cov = tg.CovarianceTransport(k, noise=None)
tgp = tg.TGP([cov], dt=None, annealing=0)

tgp.sample_priors()
tgp.plot_priors()

In [ ]:
# Se guarda el sample de cada grupo
sample_theo = tgp.get_priors(npriors=npriors, ngroups=ngroups)

### SGD

In [ ]:
learning = tg.TgLearning(tgp, lr=0.1)
learning.execute_sgd(nitersSGD)
#tgp.plot_priors(kde=False)

In [ ]:
#learning.execute_svgd(niters)
#tgp.plot_priors(kde=False)

### SVGD

In [ ]:
niters = 30 # number of iterations
nreview = 10 # number of review
rprior= 0
rgroup = 0

In [ ]:
review_dict = learning.review(niters=niters, nreview=nreview, rprior=rprior, rgroup=rgroup) 
# hay alguna forma que salga las iteraciones totales en vez de 10 en 10 ?
tgp.plot_priors(kde=False)

In [ ]:
learning.plotKS(theorical=sample_theo, review_dict=review_dict, rprior=rprior, rgroup=rgroup)